In [5]:
# CELDA 1: Imports robustos — instala matplotlib si falta y usa backend 'Agg' en entornos headless
import sys
import subprocess

# funciones de ayuda
def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# Intentar importar matplotlib — si falla, instalarlo y reimportar
try:
    import matplotlib
    # en servidores sin display preferimos 'Agg' antes de importar pyplot (evita errores de display)
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
except Exception as e:
    print("matplotlib no está instalado o falló la importación. Se intentará instalar...")
    pip_install("matplotlib")
    import importlib
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt

# Imports restantes
import pandas as pd
from collections import defaultdict
import math

print("matplotlib version:", matplotlib.__version__)
print("pandas version:", pd.__version__)


matplotlib version: 3.10.7
pandas version: 2.3.3


In [6]:
# CELDA 2: Funciones para cargar datos y completar información

def crear_data():
    """
    Lee un archivo Excel con información climatológica y crea una lista 
    de objetos `Data`. El archivo contiene cuatro columnas: año, estación 1, 
    estación 2 y mes. Las filas con valores de mes vacíos conservan el 
    último mes válido encontrado anteriormente.
    """
    df = pd.read_excel("resources/data.xlsx", usecols="A:D", header=None)
    data = []
    month = None

    for _, row in df.iterrows():
        year, station_1, station_2, month_cell = row
        if pd.isna(month_cell):
            month_cell = None
        if month_cell is not None:
            month = month_cell

        # Crear objeto Data
        data.append(Data(
            year=year,
            station_1=station_1,
            station_2=station_2,
            month=month
        ))
    return data


def completar_porcentaje_normal(data, normales):
    """
    Completa valores faltantes de precipitación en las estaciones
    utilizando el método del porcentaje normal.
    """
    data_completo = []
    for row in data:
        s1 = row.station_1
        s2 = row.station_2

        if s1 is None and s2 is not None:
            row.station_1 = (normales["station_1"] / normales["station_2"]) * s2

        if s2 is None and row.station_1 is not None:
            row.station_2 = (normales["station_2"] / normales["station_1"]) * row.station_1

        data_completo.append(row)

    return data_completo


In [7]:
# CELDA 3: Función de impresión formateada y listas de configuración

def imprimir_fila(nombre, dic, decimales=2):
    """
    Imprime una fila formateada con valores mensuales alineados.
    """
    fila = f"{nombre:<10} "
    for mes in orden_meses:
        if decimales == 5:
            fila += f"{dic[mes]:>15.5f} "
        else:
            fila += f"{dic[mes]:>10.2f} "
    print(fila)


# Lista oficial de meses en orden
orden_meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# Normales anuales de las estaciones
normales = {
    "station_1": 1200,
    "station_2": 800
}

# Lista de estaciones disponibles
estaciones = ["station_1", "station_2"]


In [8]:
# CELDA 4: Ejecución principal (versión notebook sin input)

# Ejecutar una sola vez (sin repetir)
repetir = False

# 1. Cargar y completar datos
data = crear_data()
data_completa = completar_porcentaje_normal(data, normales)

# 2. Selección de estación (SIN INPUT → por defecto station_1)
estacion = "station_1"
print(f"\nSe analizará: {estacion}\n")

# 3. Construcción de tabla año-mes
tabla = defaultdict(lambda: {mes: "" for mes in orden_meses})
for d in data_completa:
    tabla[d.year][d.month] = getattr(d, estacion)

# 4. Cálculo de estadísticas
promedios, maximos, minimos, desviaciones, pmp = {}, {}, {}, {}, {}

for mes in orden_meses:
    valores = [tabla[año][mes] for año in tabla if tabla[año][mes] != ""]
    if valores:
        n = len(valores)
        media = sum(valores) / n

        promedios[mes] = media
        maximos[mes] = max(valores)
        minimos[mes] = min(valores)

        varianza = sum((x - media) ** 2 for x in valores) / n
        desviaciones[mes] = math.sqrt(varianza)
        pmp[mes] = media + 1.5 * desviaciones[mes]
    else:
        promedios[mes] = maximos[mes] = minimos[mes] = desviaciones[mes] = pmp[mes] = 0

# 5. Impresión de tabla
titulo = f"TABLA DE PRECIPITACIÓN MENSUAL POR AÑO ({estacion.upper()})"
print(titulo.center(150, "="))
print()
print(f"{'AÑO':<10} " + " ".join(f"{m[:3]:>10}" for m in orden_meses))

for año in sorted(tabla.keys()):
    fila = f"{año:<10} "
    for mes in orden_meses:
        val = tabla[año][mes]
        val_str = f"{val:.2f}" if val != "" else "    -    "
        fila += f"{val_str:>10} "
    print(fila)

print()
imprimir_fila("MINIMO", minimos)
imprimir_fila("PROMEDIO", promedios)
imprimir_fila("MAXIMO", maximos)
imprimir_fila("DESV_STD", desviaciones, decimales=5)
imprimir_fila("PMP", pmp, decimales=5)

# 6. Suma anual multianual
suma_promedios_anual = sum(promedios[mes] for mes in orden_meses)
print(f"\nSUMA DE PROMEDIOS ANUALES MULTIANUAL: {suma_promedios_anual:.2f} mm")

# 7. Gráfica
val_min = [minimos[m] for m in orden_meses]
val_prom = [promedios[m] for m in orden_meses]
val_max = [maximos[m] for m in orden_meses]

plt.figure(figsize=(12, 6))
plt.plot(orden_meses, val_min, marker='o', linestyle='--', label='Mínimo')
plt.plot(orden_meses, val_prom, marker='s', linestyle='-', label='Promedio')
plt.plot(orden_meses, val_max, marker='^', linestyle='--', label='Máximo')

plt.title(f"Precipitación Mensual ({estacion}) - Mínimo, Promedio y Máximo")
plt.xlabel("Mes")
plt.ylabel("Precipitación (mm)")
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'resources/data.xlsx'